! pip install youtube-comment-downloader 

In [3]:
from youtube_comment_downloader import YoutubeCommentDownloader
import pandas as pd
import time 


In [ ]:
#video_id ber korte hobe 
#

In [6]:
video_ids = [
    "K0B2qlb8eVI",
    "5NLf7O6BF6s"
]

MAX_COMMENTS_PER_VIDEO = 200
SLEEP_SECONDS = 0.000001

downloader = YoutubeCommentDownloader()
all_data = []

for vid in video_ids:
    print(f"Downloading comments for video: {vid}")
    url = f"https://www.youtube.com/watch?v={vid}"

    try:
        comments = downloader.get_comments_from_url(url)
        for idx, comment in enumerate(comments, start=1):
            # Some videos/comments do not include all fields (e.g., likes).
            likes = comment.get("likes", comment.get("votes", 0))
            all_data.append({
                "video_id": vid,
                "comment": comment.get("text", ""),
                "author": comment.get("author", ""),
                "likes": likes,
                "published_at": comment.get("published_at", comment.get("time", ""))
            })

            if idx % 50 == 0:
                print(f"  Collected {idx} comments from {vid}...")

            if idx >= MAX_COMMENTS_PER_VIDEO:
                print(f"  Reached limit ({MAX_COMMENTS_PER_VIDEO}) for {vid}")
                break

            time.sleep(SLEEP_SECONDS)
    except Exception as e:
        print(f"Error downloading comments for video {vid}: {e}")

df = pd.DataFrame(all_data)
df.to_csv("youtube_comments.csv", index=False, encoding='utf-8-sig')
print(f"Comments downloaded and saved to youtube_comments.csv. Total rows: {len(df)}")


  Collected 50 comments from K0B2qlb8eVI...
  Collected 100 comments from K0B2qlb8eVI...
  Collected 150 comments from K0B2qlb8eVI...
  Collected 200 comments from K0B2qlb8eVI...
  Reached limit (200) for K0B2qlb8eVI
  Collected 50 comments from 5NLf7O6BF6s...
  Collected 100 comments from 5NLf7O6BF6s...
  Collected 150 comments from 5NLf7O6BF6s...
  Collected 200 comments from 5NLf7O6BF6s...
  Reached limit (200) for 5NLf7O6BF6s
Comments downloaded and saved to youtube_comments.csv. Total rows: 400


nlp - preprocessing 

In [7]:
! pip install  nltk spacy emoji langdetect 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.4 MB/s  0:00:0036m-:--:--
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 2.9 MB/s  0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993332 sha256=80496222a29ebea642e1bb0f0f40d924345a62e2bfb561f4db7ced38669789c6
  Stored in directory: /Users/imac/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [emoji]


In [9]:
! pip install spacy

In [12]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 113.9 kB/s  0:01:160:00:0300:05
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
# Test if spacy model is already available
import spacy
nlp = spacy.load("en_core_web_sm")
print("✓ Spacy model 'en_core_web_sm' is already installed and working!")

✓ Spacy model 'en_core_web_sm' is already installed and working!


In [14]:
import pandas as pd 
import re 
import nltk 
import spacy 
import emoji 
from langdetect import detect 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [15]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /Users/imac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/imac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df=pd.read_csv("youtube_comments.csv")
df.head()

,video_id,comment,author,likes,published_at
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে
3,K0B2qlb8eVI,So they sent the master of disaster,@andrewpetersen6116,0,15 ঘণ্টা আগে
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে


In [19]:
comment_column = None

# 1) Preferred exact/keyword matches
priority_names = {"comment", "comments", "text", "review", "message", "body"}
for col in df.columns:
    col_l = str(col).strip().lower()
    if col_l in priority_names or any(k in col_l for k in ["comment", "text", "review", "message"]):
        comment_column = col
        break

# 2) Fallback: first object/string-like column with non-empty values
if comment_column is None:
    for col in df.columns:
        series = df[col]
        if series.dtype == "object" and series.dropna().astype(str).str.strip().ne("").any():
            comment_column = col
            break

print(f"Identified comment column: {comment_column}")
if comment_column is None:
    raise ValueError(f"Could not identify a comment/text column. Available columns: {list(df.columns)}")


Identified comment column: comment


In [21]:
stop_words = set(stopwords.words('english'))
def clean_text_nlp(text):
    if pd.isna(text):
        return "","unknown"
    text=str(text)
    # try urls 
    try:
        lang=detect(text)
    except:
        lang="unknown"
    # remove emojis
    text=emoji.replace_emoji(text,replace="")
    # remove urls
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text=text.lower()
    #keep english and bangla letters 
    text = re.sub(r"[^a-zA-Z\u0980-\u09FF\s]", "", text)
    # tokenize
    tokens = word_tokenize(text)
    # remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # lemmatize
    doc = nlp(" ".join(tokens))
    lemmas=[token.lemma_ for token in doc]
    cleaned_text=" ".join(lemmas)
    return cleaned_text,lang





apply preprocessing 

In [23]:
cleaned_texts=[]
languages=[]
for text in df[comment_column]:
    cleaned,lang=clean_text_nlp(text)
    cleaned_texts.append(cleaned)
    languages.append(lang)
df["cleaned_comment"]=cleaned_texts
df["language"]=languages
df.head()

,video_id,comment,author,likes,published_at,cleaned_comment,language
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে,india powerful country,en
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে,common denominator islamist attach cause separ...,en
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে,let talk child disappear mountbatten indian le...,en
3,K0B2qlb8eVI,So they sent the master of disaster,@andrewpetersen6116,0,15 ঘণ্টা আগে,send master disaster,en
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে,muslims want pakistan kill hindus million musl...,en


remove empty and duplicants 

In [25]:
df=df[df['cleaned_comment'].str.strip()!=""]
df.head()

,video_id,comment,author,likes,published_at,cleaned_comment,language
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে,india powerful country,en
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে,common denominator islamist attach cause separ...,en
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে,let talk child disappear mountbatten indian le...,en
3,K0B2qlb8eVI,So they sent the master of disaster,@andrewpetersen6116,0,15 ঘণ্টা আগে,send master disaster,en
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে,muslims want pakistan kill hindus million musl...,en


In [26]:
df=df.drop_duplicates(subset=["cleaned_comment"])
df.head()

,video_id,comment,author,likes,published_at,cleaned_comment,language
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে,india powerful country,en
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে,common denominator islamist attach cause separ...,en
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে,let talk child disappear mountbatten indian le...,en
3,K0B2qlb8eVI,So they sent the master of disaster,@andrewpetersen6116,0,15 ঘণ্টা আগে,send master disaster,en
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে,muslims want pakistan kill hindus million musl...,en


In [27]:
df.to_csv("youtube_comments_cleaned.csv", index=False, encoding='utf-8-sig')
print(f"Cleaned comments saved to youtube_comments_cleaned.csv. Total rows: {len(df)}")

Cleaned comments saved to youtube_comments_cleaned.csv. Total rows: 391


relevant comment filter

In [28]:
keywords=[
    "india","pakistan","kasmir","modi","imran","army","border","war","peace","terrorism"
]
df["is_relevant"]=df["cleaned_comment"].apply(lambda x: any(k in x for k in keywords))
df_filtered=df[df["is_relevant"]==True]
print(f"Relevant data shape: {df_filtered.shape}")
df_filtered.head()

Relevant data shape: (213, 8)


,video_id,comment,author,likes,published_at,cleaned_comment,language,is_relevant
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে,india powerful country,en,True
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে,common denominator islamist attach cause separ...,en,True
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে,let talk child disappear mountbatten indian le...,en,True
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে,muslims want pakistan kill hindus million musl...,en,True
5,K0B2qlb8eVI,It was a master plan by the west a divided Ind...,@greggarcia9842,0,1 দিন আগে,master plan west divide india good western power,en,True


In [36]:
df_sample=df_filtered.sample(n=500, random_state=42,replace=True)
df_sample.to_csv("manual_label_data.csv", index=False, encoding='utf-8-sig')
print("Exported 500 relevant comments to manual_label_data.csv for manual labeling.")

Exported 500 relevant comments to manual_label_data.csv for manual labeling.


In [37]:
! pip install transformers torch

In [38]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

df = pd.read_csv("youtube_comments.csv")

# Ensure cleaned_comment exists even if the raw CSV has only comment/text columns.
if "cleaned_comment" not in df.columns:
    candidate_cols = [c for c in df.columns if "comment" in str(c).lower() or "text" in str(c).lower()]
    if not candidate_cols:
        raise KeyError(f"No comment/text column found. Available columns: {list(df.columns)}")
    df["cleaned_comment"] = df[candidate_cols[0]]

# Normalize and keep non-empty text only.
df["cleaned_comment"] = df["cleaned_comment"].fillna("").astype(str)
df = df[df["cleaned_comment"].str.strip() != ""].copy()
df.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


,video_id,comment,author,likes,published_at,cleaned_comment
0,K0B2qlb8eVI,India is a very powerful country,@BoazMwadime-b1k,0,6 ঘণ্টা আগে,India is a very powerful country
1,K0B2qlb8eVI,Common denominator with all Islamist is they a...,@clevelandwilliams5922,0,12 ঘণ্টা আগে,Common denominator with all Islamist is they a...
2,K0B2qlb8eVI,Let’s talk about all the children that disappe...,@PeterWright-y3b,0,15 ঘণ্টা আগে,Let’s talk about all the children that disappe...
3,K0B2qlb8eVI,So they sent the master of disaster,@andrewpetersen6116,0,15 ঘণ্টা আগে,So they sent the master of disaster
4,K0B2qlb8eVI,Muslims wanted Pakistan and killed hindus for ...,@AshishVerma-dz9ug,0,1 দিন আগে,Muslims wanted Pakistan and killed hindus for ...


In [ ]:
def auto_label(text):
    result=classifier(str(text))[0]
    label_id=result['label']
    if label_id == "LABEL_0":
        return "negative"
    elif label_id == "LABEL_1":
        return "neutral"
    else :
        return "positive"
df['label']=df['cleaned_comment'].apply(auto_label)
df.head()